In [2]:
import json
from pathlib import Path
import requests
from bs4 import BeautifulSoup

import time

from tqdm.notebook import trange, tqdm

import pandas as pd
import numpy as np

In [3]:
## 공공 데이터 API 키 설정

apiKeyPath = './api-key/public-data-gov-key.json'
apiKey = ''

with open(apiKeyPath) as json_file:
    loaded = json.load(json_file)
    apiKey = loaded['key']

In [5]:
## 법정동 코드 Pandas 로 읽기
dfDistrictCodeB = pd.read_csv('./dataset/district_code_b.csv').query("폐지여부 == False")
dfDistrictCodeB['법정구코드'] = dfDistrictCodeB['법정동코드'].apply(lambda x: int(str(x)[:5]))
print(len(dfDistrictCodeB['법정구코드'].unique()))

277


In [10]:
## 아파트 실거래가 데이터
def getAptRentResponse(code, ymd, key):
    apiPrefixGetAptTrade = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent"
    urlGetAptTrade = "{}?type=xml&LAWD_CD={}&DEAL_YMD={}&serviceKey={}".format(
        apiPrefixGetAptTrade, 
        code,
        ymd,
        key
    )

    responseApiTrade = requests.get(urlGetAptTrade, verify = False)
    return responseApiTrade.text


# 샘플 응답 포맷

getAptRentResponse(11200, '201912', apiKey)

```html
<item>
   <건축년도>2014</건축년도>
   <년>2019</년>
   <법정동> 상왕십리동</법정동>
   <보증금액>    70,000</보증금액>
   <아파트>텐즈힐(2구역)</아파트>
   <월>12</월>
   <월세금액>         0</월세금액>
   <일>16</일>
   <전용면적>84.92</전용면적>
   <지번>811</지번>
   <지역코드>11200</지역코드>
   <층>1</층>
</item>
```

In [13]:
# XML 파싱 후 필터링
def parseAptRentResponse(text):
    parsed = BeautifulSoup(text, 'lxml-xml')
    rawXmlItems = parsed.findAll("item")

    colNames = [
        '법정동',
        '지역코드',
        '아파트',
        '지번',
        '년',
        '월',
        '일',
        '건축년도',
        '전용면적',
        '층',
        '월세금액',
        '보증금액'
    ]

    dfRaw = pd.DataFrame(index = [], columns = colNames)

    # XML 내 item 을 반복하며 pandas df 로 변경
    for rawXmlItem in rawXmlItems: 
        row = dict.fromkeys(colNames)

        for col in colNames:       
            try :
                row[col] = rawXmlItem.find(col).text
            except :
                row[col] = np.nan

        dfTemp = pd.DataFrame(
            [row],
            columns = colNames)

        dfRaw = dfRaw.append(dfTemp)

    # 컬럼 타입 변경 또는 추가
    dfReturn = dfRaw
    dfReturn['거래일'] = pd.to_datetime(dfRaw['년'] + '-' + dfRaw['월'] + '-' + dfRaw['일'])
    dfReturn['월세금액'] = pd.to_numeric(dfRaw['월세금액'].str.replace(',', ''))
    dfReturn['보증금액'] = pd.to_numeric(dfRaw['보증금액'].str.replace(',', ''))

    dfReturn['지역코드'] = pd.to_numeric(dfRaw['지역코드'])
    dfReturn['년'] = pd.to_numeric(dfRaw['년'])
    dfReturn['월'] = pd.to_numeric(dfRaw['월'])
    dfReturn['일'] = pd.to_numeric(dfRaw['일'])
    
    dfReturn['법정동'] = dfRaw['법정동'].str.strip()
    dfReturn['아파트'] = dfRaw['아파트'].str.strip()
    dfReturn['지번'] = dfRaw['지번'].str.strip()

    dfReturn['건축년도'] = pd.to_numeric(dfRaw['건축년도'])
    dfReturn['전용면적'] = pd.to_numeric(dfRaw['전용면적'])
    dfReturn['층'] = pd.to_numeric(dfRaw['층'])
    
    dfReturn['p_ymd'] = pd.Timestamp("today").strftime("%Y%m%d")
    dfReturn.index = range(len(dfReturn))

    return dfReturn

In [14]:
def crawlAptRent(code, ymd, key):
    filePath = './dataset/apt-rent/{}/apt-rent-{}-{}.csv'.format(ymd, code, ymd)
    
    resAptRent = getAptRentResponse(
        code, 
        ymd,
        key)

    dfAptRent = parseAptRentResponse(resAptRent)
    
    if dfAptRent.size > 0:
        dfAptRent.to_csv(filePath, mode='w', index=False)

In [ ]:
dealYmd = '201910'
uniqueCodes = dfDistrictCodeB['법정구코드'].unique()

print('아파트 전월세 수집 ({})'.format(dealYmd))
for code in tqdm(uniqueCodes):
    
    print('수집중: {}'.format(code))
    
    time.sleep(0.10)
    
    crawlAptRent(code, dealYmd, apiKey)